In [7]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests6

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


# a  = .25, K = 19

In [9]:
a = 0.25
num_params=19
num_sims = 100

In [17]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.45 & 0.99 & 0.42 & 0.99 & 0.99 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.55 & 0.01 & 0.58 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.45, 0.  , 0.55]), array([0.99, 0.  , 0.01]), array([0.42, 0.  , 0.58]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), array([0.99, 0.01, 0.  ]), -10.44975542515146, 4.862435190447395, 5.2304582274544575)


In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.77 & 0.88 & 0.79 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.23 & 0.12 & 0.21 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.77, 0.  , 0.23]), array([0.88, 0.  , 0.12]), array([0.79, 0.  , 0.21]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -8.515402181131781, 5.221951312874477, 6.630862781174881)


In [19]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.77 & 0.77 & 0.79 & 0.97 & 0.97 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.23 & 0.23 & 0.21 & 0.02 & 0.02 & 0.02   \\
\hline
\end{tabular}
(array([0.77, 0.  , 0.23]), array([0.77, 0.  , 0.23]), array([0.79, 0.  , 0.21]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), array([0.97, 0.01, 0.02]), -10.177877989647717, 7.622346191765633, 8.701770476544555)


In [20]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.88 & 1.00 & 1.00 & 0.97   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.01   \\
Model 2 & 0.10 & 0.10 & 0.12 & 0.00 & 0.00 & 0.02   \\
\hline
\end{tabular}
(array([0.9, 0. , 0.1]), array([0.9, 0. , 0.1]), array([0.88, 0.  , 0.12]), array([1., 0., 0.]), array([1., 0., 0.]), array([0.97, 0.01, 0.02]), -8.05213453930057, 10.621325111200084, 11.41130292193911)


# a  = .25, k= 4, something not right?

In [24]:
a = 0.25
num_params=4
num_sims = 100

In [25]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.96 & 1.00 & 1.00 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.04 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), array([0.96, 0.  , 0.04]), array([1., 0., 0.]), array([1., 0., 0.]), array([1., 0., 0.]), -2.249267922748295, 5.2393809035209395, 5.723408698574368)


In [26]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.97 & 0.97 & 0.98 & 0.98 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.03 & 0.03 & 0.03 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), array([0.97, 0.  , 0.03]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.99, 0.  , 0.01]), -2.5471521184182824, 7.478193080535721, 7.734398536438539)


# a  = .25, k= 9

In [27]:
a = 0.25
num_params=9
num_sims = 100

In [28]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.86 & 0.86 & 0.88 & 0.91 & 0.91 & 0.96   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.06 & 0.06 & 0.03   \\
Model 2 & 0.14 & 0.14 & 0.12 & 0.03 & 0.03 & 0.01   \\
\hline
\end{tabular}
(array([0.86, 0.  , 0.14]), array([0.86, 0.  , 0.14]), array([0.88, 0.  , 0.12]), array([0.91, 0.06, 0.03]), array([0.91, 0.06, 0.03]), array([0.96, 0.03, 0.01]), -3.8968088130803453, 6.6316762402314335, 5.937942956746057)


In [29]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.98 & 0.95 & 0.98 & 0.98 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.01 & 0.01 & 0.01 & 0.01   \\
Model 2 & 0.02 & 0.02 & 0.04 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.98, 0.  , 0.02]), array([0.98, 0.  , 0.02]), array([0.95, 0.01, 0.04]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), -3.903703712222221, 6.291603787966217, 7.887010178631539)


In [30]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.96 & 0.91 & 0.97 & 0.97 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.02 & 0.02 & 0.01   \\
Model 2 & 0.11 & 0.04 & 0.09 & 0.01 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.89, 0.  , 0.11]), array([0.96, 0.  , 0.04]), array([0.91, 0.  , 0.09]), array([0.97, 0.02, 0.01]), array([0.97, 0.02, 0.01]), array([0.98, 0.01, 0.01]), -4.410367699484034, 3.943549815067037, 4.475365964249693)


# evidence of power

In [31]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9
num_sims = 100

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [32]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.90 & 0.90 & 0.90 & 0.51 & 0.51 & 0.57   \\
Model 1 & 0.10 & 0.10 & 0.10 & 0.49 & 0.49 & 0.43   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.9, 0.1, 0. ]), array([0.9, 0.1, 0. ]), array([0.9, 0.1, 0. ]), array([0.51, 0.49, 0.  ]), array([0.51, 0.49, 0.  ]), array([0.57, 0.43, 0.  ]), 6.314053966132174, 4.578263343008387, 5.388472722496302)


In [33]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.81 & 0.82 & 0.80 & 0.48 & 0.48 & 0.56   \\
Model 1 & 0.19 & 0.18 & 0.20 & 0.52 & 0.52 & 0.44   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.81, 0.19, 0.  ]), array([0.82, 0.18, 0.  ]), array([0.8, 0.2, 0. ]), array([0.48, 0.52, 0.  ]), array([0.48, 0.52, 0.  ]), array([0.56, 0.44, 0.  ]), 6.713671099566486, 4.950672328864715, 5.2935181621946175)


In [34]:
a1,a2 = 0.00,np.sqrt(1.09-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.08 & 0.19 & 0.07 & 0.50 & 0.50 & 0.56   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.92 & 0.81 & 0.93 & 0.50 & 0.50 & 0.44   \\
\hline
\end{tabular}
(array([0.08, 0.  , 0.92]), array([0.19, 0.  , 0.81]), array([0.07, 0.  , 0.93]), array([0.5, 0. , 0.5]), array([0.5, 0. , 0.5]), array([0.56, 0.  , 0.44]), -14.849687343944911, 5.2753080483192125, 5.320207048492602)


In [35]:
a1,a2 =  0.00,np.sqrt(1.09**.5-1)
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests6.monte_carlo(num_sims,gen_data_ex,setup_shi_ex)
vuong_tests6.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.10 & 0.21 & 0.13 & 0.61 & 0.62 & 0.69   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.90 & 0.79 & 0.87 & 0.39 & 0.38 & 0.31   \\
\hline
\end{tabular}
(array([0.1, 0. , 0.9]), array([0.21, 0.  , 0.79]), array([0.13, 0.  , 0.87]), array([0.61, 0.  , 0.39]), array([0.62, 0.  , 0.38]), array([0.69, 0.  , 0.31]), -13.958286829803896, 4.982295664680166, 5.249750987845085)
